## Import libraries

In [1]:
import pandas as pd
import os

## Functions

### Set directory

In [2]:
# Ver el directorio de trabajo actual
current_directory = os.getcwd()
print(f"Directorio de trabajo actual: {current_directory}")

Directorio de trabajo actual: C:\Users\famwa\Tesina\NBA\Scripts\ETL


In [3]:
# Cambiar el directorio de trabajo
#new_directory = r"C:\Users\famwa\Tesina\NBA"
new_directory = r"C:\Users\famwa\Tesina\NBA"
os.chdir(new_directory)

# Verificar que el directorio de trabajo ha cambiado
current_directory = os.getcwd()
print(f"Directorio de trabajo actual: {current_directory}")

Directorio de trabajo actual: C:\Users\famwa\Tesina\NBA


### Read and drop NaNs

In [4]:
def merge_and_clean_datasets(folder_name, file_name, start_year, end_year, threshold=0.3):
    """
    Esta función combina varios archivos CSV en un solo DataFrame y luego elimina las columnas con una proporción de valores NaN, Null o None mayor que un umbral dado.
    Cada archivo debe tener el formato 'file_name_year.csv' y estar ubicado en la carpeta especificada dentro de la carpeta 'Data'.
    Se añade una columna 'year' al DataFrame final basada en el nombre del archivo.

    Parámetros:
    folder_name : str
        El nombre de la carpeta que contiene los archivos a combinar.
    file_name : str
        El nombre común de los archivos a combinar.
    start_year : int
        El primer año en el rango de archivos a combinar.
    end_year : int
        El último año en el rango de archivos a combinar.
    threshold : float, opcional
        El umbral de proporción de valores NaN, Null o None para eliminar una columna.

    Devuelve:
    df : DataFrame
        Un DataFrame que combina todos los archivos de entrada y con las columnas con exceso de valores NaN, Null o None eliminadas.
    """

    # Lista para almacenar los dataframes
    df_list = []

    for year in range(start_year, end_year + 1):
        # Crear el nombre del archivo
        file = os.path.join("Data", folder_name, f"{file_name}_{year}.csv")

        # Leer el archivo CSV en un DataFrame
        df = pd.read_csv(file)

        # Añadir una columna 'year' al DataFrame
        df['Year'] = year

        # Añadir el DataFrame a la lista
        df_list.append(df)

    # Concatenar todos los DataFrames en la lista
    merged_df = pd.concat(df_list, ignore_index=True)

    # Calcular la proporción de valores NaN, Null o None en cada columna
    nan_ratio = merged_df.isnull().mean()

    # Encontrar las columnas que tienen una proporción de valores NaN, Null o None mayor que el umbral
    columns_to_remove = nan_ratio[nan_ratio > threshold].index

    # Eliminar las columnas
    merged_df = merged_df.drop(columns_to_remove, axis=1)
    
    # Guardar el DataFrame resultante en un archivo CSV
    merged_df.to_csv(f"Data/Panel/{file_name}_panel.csv", index=False)

    return merged_df

### Drop columns

In [5]:
def drop_columns(df, columns_to_drop):
    """
    Esta función toma un DataFrame y una lista de nombres de columnas, y devuelve un nuevo DataFrame que no incluye las columnas especificadas.

    Parámetros:
    df : DataFrame
        El DataFrame original.
    columns_to_drop : list
        Una lista de nombres de columnas a eliminar.

    Devuelve:
    df : DataFrame
        Un nuevo DataFrame que no incluye las columnas especificadas.
    """

    df = df.drop(columns=columns_to_drop)
    
    return df

### Strings to numeric value

In [6]:
def convert_strings_to_numeric(df, exclude_columns=[]):
    """
    Esta función toma un DataFrame y convierte todas las cadenas que representan números o cantidades monetarias en formato float.
    Las cadenas deben tener el formato 'xxx,xxx.xx', 'xxxxx.xx', 'xxxxxxx' o '$xxxx,xxx.xx', donde 'x' es un dígito.
    Las columnas especificadas en 'exclude_columns' no serán transformadas.
    Las columnas que contienen solo cadenas que no representan números también son excluidas.

    Parámetros:
    df : DataFrame
        El DataFrame a convertir.
    exclude_columns : list, opcional
        Una lista de nombres de columnas que no deben ser transformadas.

    Devuelve:
    df : DataFrame
        El DataFrame con las cadenas convertidas a float.
    """

    def convert_string(s):
        """
        Esta función auxiliar convierte una cadena que representa un número o una cantidad monetaria en un float.
        Si la cadena no representa un número o una cantidad monetaria, se devuelve el valor original.
        """
        try:
            # Eliminar el símbolo del dólar y las comas, y convertir a int
            return int(str(s).replace('$', '').replace(',', ''))
        except ValueError:
            try:
                # Si la cadena no puede ser convertida a int, intentar convertir a float
                return float(str(s).replace('$', '').replace(',', ''))
            except ValueError:
                try:
                    # Si la cadena no puede ser convertida a float, intentar convertir a numérico con pd.to_numeric
                    return pd.to_numeric(s, errors='coerce')
                except ValueError:
                    # Si la cadena no puede ser convertida a numérico, devolver el valor original
                    return s

    # Aplicar la función de conversión a cada elemento del DataFrame, excepto en las columnas excluidas
    for col in df.columns:
        if col not in exclude_columns and df[col].dtype == 'object':
            # Verificar si todos los elementos de la columna son cadenas que representan números
            if df[col].apply(lambda x: isinstance(x, str) and (x.replace('.', '', 1).isdigit() or ('$' in x and x.replace('$', '').replace(',', '').replace('.', '', 1).isdigit()))).all():
                df[col] = df[col].apply(convert_string)
            else:
                exclude_columns.append(col)

    return df

### From numeric into string

In [7]:
def convert_to_string(df, columns_to_convert):
    """
    Esta función toma un DataFrame y una lista de nombres de columnas, y convierte las columnas especificadas a tipo string.

    Parámetros:
    df : DataFrame
        El DataFrame a convertir.
    columns_to_convert : list
        Una lista de nombres de columnas a convertir a string.

    Devuelve:
    df : DataFrame
        El DataFrame con las columnas convertidas a string.
    """

    for col in columns_to_convert:
        df[col] = df[col].astype(str)

    return df

### Square values

In [8]:
def square_numeric_columns(df):
    """
    Esta función toma un DataFrame y calcula el cuadrado de todas las columnas numéricas (float o int).
    Se añade una nueva columna al DataFrame para cada columna numérica, que contiene el cuadrado de los valores de la columna original.
    El nombre de la nueva columna es el nombre de la columna original con el sufijo '_2'.

    Parámetros:
    df : DataFrame
        El DataFrame a transformar.

    Devuelve:
    df : DataFrame
        El DataFrame con las nuevas columnas añadidas.
    """

    # Obtener una lista de las columnas numéricas
    numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns

    # Para cada columna numérica, calcular el cuadrado de los valores y añadir una nueva columna al DataFrame
    for col in numeric_cols:
        df[col + '_2'] = df[col] ** 2

    return df

### Sort columns

In [9]:
def sort_columns(df, first_columns):
    """
    Esta función toma un DataFrame y una lista de nombres de columnas, y ordena las columnas del DataFrame alfabéticamente.
    Las columnas especificadas en 'first_columns' se mantienen al principio.

    Parámetros:
    df : DataFrame
        El DataFrame a ordenar.
    first_columns : list
        Una lista de nombres de columnas a mantener al principio.

    Devuelve:
    df : DataFrame
        El DataFrame con las columnas ordenadas.
    """

    # Obtener una lista de todas las columnas del DataFrame
    all_columns = list(df.columns)

    # Crear una lista de las columnas que no están en 'first_columns'
    other_columns = [col for col in all_columns if col not in first_columns]

    # Ordenar alfabéticamente las 'other_columns'
    other_columns.sort()

    # Crear una nueva lista de columnas que comienza con 'first_columns' y continúa con 'other_columns'
    sorted_columns = first_columns + other_columns

    # Reordenar las columnas del DataFrame
    df = df[sorted_columns]

    return df

### Add t sufix

In [10]:
def add_suffix_to_numeric_columns(df):
    """
    Esta función agrega el sufijo '_t' a todas las columnas numéricas en un DataFrame de pandas.

    Parámetros:
    df : DataFrame
        El DataFrame a procesar.

    Devuelve:
    df : DataFrame
        El DataFrame procesado con nombres de columnas modificados.
    """
    for col in df.columns:
        if pd.api.types.is_numeric_dtype(df[col]):
            df = df.rename(columns={col: col + '_t'})

    return df

### Get data of previous year

In [11]:
def add_previous_year_columns(df):
    """
    Esta función agrega columnas para el valor del año anterior para cada columna numérica con sufijo '_t' en un DataFrame de pandas.

    Parámetros:
    df : DataFrame
        El DataFrame a procesar.

    Devuelve:
    df : DataFrame
        El DataFrame procesado con columnas adicionales para los valores del año anterior.
    """

    # Asegúrate de que el DataFrame esté ordenado por año
    df = df.sort_values('Year')

    # Para cada columna en el DataFrame
    for col in df.columns:
        # Si la columna es numérica y tiene el sufijo '_t'
        if pd.api.types.is_numeric_dtype(df[col]) and col.endswith('_t'):
            # Crea una nueva columna con el sufijo '_t_1', que es la columna original desplazada hacia abajo por una fila
            df[col.replace('_t', '_t_1')] = df[col].shift(1)

    return df

### Aplicación de las funciones anteriores

In [12]:
def clean_process_dataframe(df, columns_to_drop=[], columns_to_convert=[], exclude_columns=[], first_columns=[], square_param='No', suffix_param='No', previous_year_param='No'):
    """
    Esta función toma un DataFrame y realiza varias transformaciones:
    1. Elimina las columnas especificadas.
    2. Convierte las cadenas que representan números o cantidades monetarias en las columnas especificadas a formato float.
    3. Convierte las columnas especificadas a tipo string.
    4. Si el parámetro square_param es 'Si', calcula el cuadrado de todas las columnas numéricas y añade una nueva columna al DataFrame para cada columna numérica.
    5. Ordena alfabéticamente las columnas del dataframe salvo un conjunto de columnas que se colocan al principio.
    6. Si el parámetro suffix_param es 'Si', añade el sufijo '_t' a todas las columnas numéricas.
    7. Si el parámetro previous_year_param es 'Si', añade columnas para el valor del año anterior para cada columna numérica con sufijo '_t'.

    Parámetros:
    df : DataFrame
        El DataFrame a procesar.
    columns_to_drop : list, opcional
        Una lista de nombres de columnas a eliminar.
    columns_to_convert : list, opcional
        Una lista de nombres de columnas a convertir a string.
    exclude_columns : list, opcional
        Una lista de nombres de columnas que no deben ser transformadas.
    first_columns : list, opcional
        Una lista de nombres de columnas que se colocarán al principio del dataframe.
    square_param : str, opcional
        Si es 'Si', se ejecuta la función square_numeric_columns. Si es cualquier otro valor, no se ejecuta.
    suffix_param : str, opcional
        Si es 'Si', se ejecuta la función add_suffix_to_numeric_columns. Si es cualquier otro valor, no se ejecuta.
    previous_year_param : str, opcional
        Si es 'Si', se ejecuta la función add_previous_year_columns. Si es cualquier otro valor, no se ejecuta.

    Devuelve:
    df : DataFrame
        El DataFrame procesado.
    """

    # Eliminar las columnas especificadas
    df = drop_columns(df, columns_to_drop)

    # Convertir las cadenas que representan números o cantidades monetarias a formato float
    df = convert_strings_to_numeric(df, exclude_columns)

    # Convertir las columnas especificadas a tipo string
    df = convert_to_string(df, columns_to_convert)

    # Calcular el cuadrado de todas las columnas numéricas si square_param es 'Si'
    if square_param.lower() == 'si':
        df = square_numeric_columns(df)
    
    # Ordena las columnas de acuerdo a las especificaciones
    df = sort_columns(df, first_columns)

    # Añade el sufijo '_t' a todas las columnas numéricas si suffix_param es 'Si'
    if suffix_param.lower() == 'si':
        df = add_suffix_to_numeric_columns(df)

    # Añade columnas para el valor del año anterior para cada columna numérica con sufijo '_t' si previous_year_param es 'Si'
    if previous_year_param.lower() == 'si':
        df = add_previous_year_columns(df)

    return df

## Aplicación del código

1. Lectura, fusión y eliminación de columnas

In [25]:
# Data de agentes libres
free_agent_panel = merge_and_clean_datasets("Free_Agent", "fa", 2011, 2022)
# Data de los salarios de los equipos de free agent
free_agent_team_salary_panel = merge_and_clean_datasets("Free_Agent_Team_Salary", "team_salary", 2011, 2022)
# Data de los contratos por temporada
salary_panel = merge_and_clean_datasets("Salary", "contracts", 2011, 2022)
# Data de las stats de los jugadores
player_stats_panel = merge_and_clean_datasets("Stats", "player_stat", 2011, 2022)
# Data de los equipos por temporada: wins, losses, etc
teams_panel = merge_and_clean_datasets("Teams", "team_victories", 2011, 2022)

#stats_panel = merge_and_clean_datasets("Stats", "stats", 2011, 2022)

,team,yearSeason,nteam,winsTeam,lossesTeam,pctWins,EBITDA,Value,Win.to.Player_Cost_Ratio,Year
0,ATL,2011,Atlanta Hawks,44,38,0.537,NaN,NaN,NaN,2011
1,BOS,2011,Boston Celtics,56,26,0.683,NaN,NaN,NaN,2011
2,BRK,2011,Brooklyn Nets,24,58,0.293,NaN,NaN,NaN,2011
3,CHA,2011,Charlotte Hornets,34,48,0.415,NaN,NaN,NaN,2011
4,CHI,2011,Chicago Bulls,62,20,0.756,NaN,NaN,NaN,2011
...,...,...,...,...,...,...,...,...,...,...
355,SAC,2022,Sacramento Kings,30,52,0.366,29M,2B,58.0,2022
356,SAS,2022,San Antonio Spurs,34,48,0.415,99M,2B,78.0,2022
357,TOR,2022,Toronto Raptors,48,34,0.585,86M,3.1B,103.0,2022
358,UTA,2022,Utah Jazz,49,33,0.598,58M,2B,95.0,2022


2. Lectura de páneles limpios

Tiene que ir a la carpeta donde se guardaron y cambiar a mano el formato de número a "General" para que funcionen apropiadamente el resto de algoritmos

In [28]:
free_agent_panel = pd.read_csv("Data/Panel/fa_panel.csv")
stats_panel = pd.read_csv("Data/Panel/stats_panel.csv")

3. Aplicación del algoritmo que llama a otras funciones

In [29]:
#columns to drop= 'Cash2023'
stats_panel = clean_process_dataframe(stats_panel, [], ['Year'], ['Year'], ['Player', 'Year', 'Pos', 'Team'], 'Si', 'Si', 'Si')

In [30]:
free_agent_panel = clean_process_dataframe(free_agent_panel, [], ['Year'], ['Year'], ['Player', 'Year', 'Pos'])

## Verificación de los resultados

In [32]:
stats_panel.head()

,Player,Year,Pos,Team,3PTM_t,3PTM_2_t,AST_t,AST_2_t,Age_t,Age_2_t,...,OREB_t_1,OREB_2_t_1,STL_t_1,STL_2_t_1,TOV_t_1,TOV_2_t_1,TVS_t_1,TVS_2_t_1,Weight_t_1,Weight_2_t_1
0,Xavier Silas,2011,SG,PHI,2,4,3,9,24,576,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
131,James Harden,2011,SG,OKC,120,14400,236,55696,22,484,...,2.0,4.0,0.0,0.0,3.0,9.0,0.00,0.0000,198.0,39204.0
132,Kevin Love,2011,C,MIN,107,11449,114,12996,23,529,...,32.0,1024.0,63.0,3969.0,144.0,20736.0,96.25,9264.0625,220.0,48400.0
133,Wesley Matthews,2011,SG,POR,133,17689,115,13225,25,625,...,226.0,51076.0,46.0,2116.0,127.0,16129.0,99.97,9994.0009,251.0,63001.0
134,Evan Turner,2011,SF,PHI,12,144,187,34969,23,529,...,57.0,3249.0,99.0,9801.0,76.0,5776.0,60.47,3656.6209,218.0,47524.0


In [31]:
free_agent_panel.head()

,Player,Year,Pos,AAV,GTDAT SIGN,GTDPRACTICAL,Status,Team From,Team From To,Value,YR-1 CAP,YR-1 CASH,YRS
0,Marc Gasol,2011,C,14375941,0,0,RFA,MEM,MEM,57503764,0,0,4
1,Tyson Chandler,2011,C,13852363,0,0,UFA,DAL,NYK,55409450,0,0,4
2,Thaddeus Young,2011,PF,8800000,0,0,RFA,PHI,PHI,43999998,0,0,5
3,DeAndre Jordan,2011,C,10759764,0,0,RFA,LAC,LAC,43039054,0,0,4
4,Arron Afflalo,2011,SG,7342500,0,36712500,RFA,DEN,DEN,36712500,0,0,5


In [33]:
# Supongamos que 'df' es tu DataFrame
print(stats_panel.dtypes)

Player           object
Year             object
Pos              object
Team             object
3PTM_t            int64
                 ...   
TOV_2_t_1       float64
TVS_t_1         float64
TVS_2_t_1       float64
Weight_t_1      float64
Weight_2_t_1    float64
Length: 72, dtype: object


In [34]:
# Supongamos que 'df' es tu DataFrame
print(free_agent_panel.dtypes)

Player          object
Year            object
Pos             object
AAV              int64
GTDAT SIGN       int64
GTDPRACTICAL     int64
Status          object
Team From       object
Team From To    object
Value            int64
YR-1 CAP         int64
YR-1 CASH        int64
YRS              int64
dtype: object


# Supongamos que 'df' es tu DataFrame
print(free_agent_panel.dtypes)

In [ ]:
# Supongamos que 'df' es tu DataFrame
nan_counts = stats_panel.isnull().sum()
print(nan_counts)

In [ ]:
# Supongamos que 'df' es tu DataFrame
nan_counts = free_agent_panel.isnull().sum()
print(nan_counts)